In [ ]:
import numpy as np
import cv2

MHI_DURATION = 0.5
DEFAULT_THRESHOLD = 32
MAX_TIME_DELTA = 0.25
MIN_TIME_DELTA = 0.05

# (empty) trackbar callback
def nothing(dummy):
    pass

def draw_motion_comp(vis, rect, angle, color):
    # 画矩形
    x, y, w, h = rect
    cv2.rectangle(vis, (x, y), (x+w, y+h), (0, 255, 0))

    # 从矩阵中心点，往 angle 方向画一条直线
    cx, cy = x + w//2, y + h//2
    angle = angle * np.pi / 180
    r = min(w//2, h//2)
    cv2.line(vis, (cx, cy), (int(cx+np.cos(angle)*r), int(cy+np.sin(angle)*r)), color, 2)


cv2.namedWindow('motempl')

cam = cv2.VideoCapture('./video/vtest.avi')

visuals = ['input', 'frame_diff', 'motion_hist']
cv2.createTrackbar('visual', 'motempl', 2, len(visuals)-1, nothing)
cv2.createTrackbar('threshold', 'motempl', DEFAULT_THRESHOLD, 255, nothing)

ret, frame = cam.read()
prev_frame = frame.copy()
motion_history = np.zeros(frame.shape[0:2], dtype=np.float32)

while True:
    ret, now_frame = cam.read()
    if ret == False:
        break

    frame_diff = cv2.absdiff(now_frame, prev_frame)
    gray_diff = cv2.cvtColor(frame_diff, cv2.COLOR_BGR2GRAY)

    thrs = cv2.getTrackbarPos('threshold', 'motempl')
    ret, motion_mask = cv2.threshold(gray_diff, DEFAULT_THRESHOLD, 1, cv2.THRESH_BINARY)

    # 这里是 motempl 关键：直接无脑按照顺序用就行
    timestamp = cv2.getTickCount() / cv2.getTickFrequency()
    cv2.motempl.updateMotionHistory(motion_mask, motion_history, timestamp, MHI_DURATION)
    mg_mask, mg_orient = cv2.motempl.calcMotionGradient(motion_history, MAX_TIME_DELTA, MIN_TIME_DELTA, apertureSize=5)
    seg_mask, seg_bounds = cv2.motempl.segmentMotion(motion_history, timestamp, MAX_TIME_DELTA)

    # 这里和 motempl 无关，功能相当于选择显示的画面
    vis_name = visuals[cv2.getTrackbarPos('visual', 'motempl')]
    if vis_name == 'input':
        vis_frame = now_frame.copy()
    elif vis_name == 'frame_diff':
        vis_frame = frame_diff.copy()
    elif vis_name == 'motion_hist':
        vis_frame = np.uint8(np.clip((motion_history-(timestamp-MHI_DURATION)) / MHI_DURATION, 0, 1)*255)
        vis_frame = cv2.cvtColor(vis_frame, cv2.COLOR_GRAY2BGR)


    # 这也是 motempl 关键：计算一些点的角度信息
    for i, rect in enumerate(list(seg_bounds)):
        x, y, rw, rh = rect
        area = rw*rh
        if area < 64**2:
            continue

        silh_roi   = motion_mask   [y:y+rh,x:x+rw]
        orient_roi = mg_orient     [y:y+rh,x:x+rw]
        mask_roi   = mg_mask       [y:y+rh,x:x+rw]
        mhi_roi    = motion_history[y:y+rh,x:x+rw]
        # NORM_L1：本质上就是矩阵的绝对值和；相当于 mask 为 1 的数量要大于 5%
        if cv2.norm(silh_roi, cv2.NORM_L1) < area * 0.05:
            continue

        angle = cv2.motempl.calcGlobalOrientation(orient_roi, mask_roi, mhi_roi, timestamp, MHI_DURATION)
        draw_motion_comp(vis_frame, rect, angle, (255,0,0))
    cv2.imshow('motempl', vis_frame)

    prev_frame = np.copy(now_frame)
    cv2.waitKey(10)

cam.release()
cv2.destroyAllWindows()